In [1]:
#%matplotlib widget
import matplotlib.pyplot as plt 

import ipywidgets as widgets
import numpy as np
import os
import pickle
import sys, math
from glob import glob

import colour
from colour_hdri.plotting import plot_tonemapping_operator_image
colour.plotting.colour_style()
colour.utilities.describe_environment();

#import cv2 as cv
import skimage
import imageio

from astropy.io import fits
from colour.models import RGB_COLOURSPACES, RGB_luminance

colorspace = RGB_COLOURSPACES['sRGB']

import astrobf
from astrobf.utils import mask_utils
from astrobf.utils.mask_utils import *

from matplotlib import patches
from matplotlib.collections import PatchCollection

import statmorph
import re


/home/hoseung/.local/lib/python3.8/site-packages/colour/utilities/verbose.py:237: ColourWarning: "Adobe DNG Converter" is not available on your platform!
  warn(*args, **kwargs)


*                                                                             *
*   Interpreter :                                                             *
*       python : 3.8.5 (default, Sep  4 2020, 07:30:14)                       *
*                [GCC 7.3.0]                                                  *
*                                                                             *
*   colour-science.org :                                                      *
*       colour : 0.3.16                                                       *
*       colour-hdri : 0.1.8                                                   *
*                                                                             *
*   Runtime :                                                                 *
*       imageio : 2.9.0                                                       *
*       matplotlib : 3.3.2                                                    *
*       networkx : 2.5                  

In [2]:
dataset = ['EFIFI','Nair'][1]
basedir = ['../../bf_data/EFIGI_catalog/','../../bf_data/Nair_and_Abraham_2010/'][1]
fitsdir = basedir + ['fits_temp_Jan_19/','fits_temp_Dec_28/', 'fits_temp_Feb_3/'][2]


#wdir = '../../OBSdata/efigi-1.6/ima_r/'
fns_g = glob(fitsdir+"*/*g.fits")
fns_r = glob(fitsdir+"*/*r.fits")
fns_i = glob(fitsdir+"*/*i.fits")

fns_g.sort()
fns_r.sort()
fns_i.sort()

eps = 1e-6

print(len(fns_r))

6431


# 2. Build a mask (GMM)

In [3]:
def extract_int(g_path):
    return int(re.split('(\d+)',g_path.split('/')[-2])[1])

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# StatMorph

## Need Image, Mask, Weight, and PSF

In [4]:
import re

fields = [
    'xc_centroid', 'yc_centroid', 
    'ellipticity_centroid', 'elongation_centroid', 'orientation_centroid',
    'xc_asymmetry', 'yc_asymmetry',
    'ellipticity_asymmetry', 'elongation_asymmetry', 'orientation_asymmetry',
    'rpetro_circ', 'rpetro_ellip', 'rhalf_circ', 'rhalf_ellip',
    'r20', 'r80',
    'gini', 'm20', 'gini_m20_bulge', 'gini_m20_merger',
    'sn_per_pixel', 'concentration', 'asymmetry', 'smoothness',
    'sersic_xc', 'sersic_yc', 'sersic_amplitude', 'sersic_rhalf', 'sersic_n', 'sersic_ellip', 'sersic_theta',
    'sky_mean', 'sky_median', 'sky_sigma',
    'flag', 'flag_sersic']

In [5]:
# Note that there is degeneracy between parameters. 
# Best fit model from scipy.iptimize of the Uncharted TMO is quite different. 
# as [   1.10147711    0.99881915  340.77234402    0.99337089].

def Mantiuk_Seidel(lum, b, c, dl, dh):
    al = (c*dl-1)/dl # contrast compression for shadows
    ah = (c*dh-1)/dh
    lp = np.log10(lum) # L prime

    conditions=[lp <= b-dl,
                (b-dl < lp) * (lp <= b),
                (b < lp) * (lp <= b+dh),
                lp > b+dh]

    functions=[0,
               lambda lp : 1/2*c*(lp-b)/(1-al*(lp-b))+1/2,
               lambda lp : 1/2*c*(lp-b)/(1+ah*(lp-b))+1/2,
               1]

    return np.piecewise(lp, conditions, functions)

params = [[1.68, 0.53, 4.12, 2.46],
          [1.90, 0.88, 2.43, 1.48],
          [1.30, 0.69, 2.53, 7.60],
          [1.38, 0.71, 1.99, 2.10]]

tmo_names = ['BriAda', 'Ashik', 'Retinex', 'log']

In [6]:
# Good galaxies after visual inspection
good_list = glob(basedir+'out1/'+"stat_png/good/*.png")
good_gids=[]
for gg in good_list:
    good_gids.append(gg.split("/")[-1].split("_")[1])
good_gids.sort()
good_fns = [glob(fitsdir+gg+'/*r.fits')[0] for gg in good_gids]

# Todo

- Reshape / organize data to minimize IO
- TMO as a class. Initialize only once for a set of params. 



In [7]:
from statmorph.utils import image_diagnostics
#import make_figure

def get_fn(gid_string):
    return glob('../../bf_data/Nair_and_Abraham_2010/fits_temp_Feb_3/' + gid_string + '/*-r.fits')[0]

def gen_fn_img(morph):
    return out_dir+f'stat_png/final_{morph._gid}_summary.png'

The meaning of the paramters of a generic TM function (including both MS08 and one from NVIDIA) depends on the range of pixel values. A straightforward scaling in the preprocessing is needed. Do I want to include that as another paramter to the BO loop? probably yes, but,next time.

In [8]:
import logging
#tmo_params = {'a':0.125, 'd':0.975, 'mid_in':0.25, 'mid_out':0.18}
logging.basicConfig(filename='applyting_MS08.log', 
                    format='%(asctime)s - %(message)s', level=logging.INFO)

for i, (tmo_name, tmo_params) in enumerate(zip(tmo_names,params)):
    if i < 3:
        continue
    print(tmo_name, tmo_params)
    out_dir = basedir+tmo_name+'/'
    if not os.path.isdir(out_dir): 
        os.mkdir(out_dir)
        os.mkdir(out_dir+'Morphs')
        os.mkdir(out_dir+'stat_png')
    # mkdir
    morphs=[]
    for i, fn in enumerate(good_fns):
        if i < 0:
            continue
        if i % 500 == 499:
            pickle.dump(morphs, open(out_dir+f"Morphs/final_morphs{i:05d}.pickle", "wb"),
                       protocol=pickle.HIGHEST_PROTOCOL)
            morphs=[]
            print(i)

        img_name = fn.split("/")[-2]
        if dataset=="Nair": 
            gid = img_name
        else:
            gid = int(re.split('(\d+)',img_name)[1])
        hdulist = fits.open(fn)
        # Ensure pixel values are positive
        img = hdulist[0].data
        img -= (img.min() - eps) 
        
        mask = pickle.load(open(basedir+'out1/'+f"/masks/{img_name}_mask.pickle", 'rb'))

        weight = fits.open(fn.replace(".fits", ".weight.fits"))[0].data
        subtracted = img.copy()
        subtracted[~mask] = np.nan
        
        #hdr_max = np.percentile(subtracted.ravel(), 99.9) * 1.1 
        subtracted *= 100 # MS08's generic TMs work best for pixels in (1e-2, 1e4)
        
        #print('min max', subtracted.min(), subtracted.max())
        #tonemapped = tonemapping_operator_generic(subtracted, *tmo_params)
        tonemapped = Mantiuk_Seidel(subtracted, *tmo_params)
        logging.info('subtrated min max: {}, {} --  tonemapped min max: {},{}'.format(subtracted.min(), subtracted.max(), tonemapped.min(), tonemapped.max()))
        morph = statmorph.source_morphology(tonemapped, mask, weightmap=weight, sersic_maxiter=0)[0]

        hdulist.close()
        morph._gid = gid
        
        statmorph.utils.image_diagnostics.make_figure(morph, nrows=3,
                                      savefig=gen_fn_img(morph),
                                      img_org=None, norm='linear')
        
        morphs.append(morph)
    pickle.dump(morphs, open(out_dir+f"Morphs/final_morphs{i:05d}.pickle", "wb"),
                protocol=pickle.HIGHEST_PROTOCOL)

log [1.38, 0.71, 1.99, 2.1]


499


/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)


/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/hom

/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)


...

/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)


3999


/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)
/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)


/home/hoseung/Work/tonemap/statmorph/statmorph/statmorph.py:506: RuntimeWarning: invalid value encountered in sqrt
  local_std = np.sqrt(local_mean2 - local_mean**2)


4499


In [8]:
all_morphs

['../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs00499.pickle',
 '../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs00999.pickle',
 '../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs01499.pickle',
 '../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs01999.pickle',
 '../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs02499.pickle',
 '../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs02999.pickle',
 '../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs03499.pickle',
 '../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs03999.pickle',
 '../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs04499.pickle',
 '../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs04999.pickle',
 '../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs05499.pickle',
 '../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs05999.pickle',
 '../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs06430.pickle']

In [125]:
out_dir

'../../bf_data/Nair_and_Abraham_2010/BriAda/'

In [16]:
for alm in all_morphs:
    print(alm)
    mps = pickle.load(open(alm,'rb'))
    for morph in mps:
        img_name = morph._gid
        fn = get_fn(img_name)
        # Double Check
            
        hdulist = fits.open(fn)
        img = hdulist[0].data
        img -= (img.min() - eps)
        
        image_diagnostics.make_figure(morph, nrows=3,
                                      savefig=gen_fn_img(morph),
                                      img_org=None)

../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs05499.pickle


/home/hoseung/anaconda3/envs/tm38/lib/python3.8/site-packages/astropy/modeling/functional_models.py:2450: RuntimeWarning: overflow encountered in power
  return amplitude * np.exp(-bn * (z ** (1 / n) - 1))


../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs05999.pickle
../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs06430.pickle


## Move bad measurements 

In [21]:
import shutil

In [25]:
for alm in all_morphs:
    print(alm)
    mps = pickle.load(open(alm,'rb'))
    for morph in mps:
        if morph.flag ==1:
            fn_img = gen_fn_img(morph)
            try:
                shutil.move(fn_img, fn_img.replace('stat_png/', 'stat_png/bad/'))
            except:
                pass

../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs00499.pickle
../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs00999.pickle
../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs01499.pickle
../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs01999.pickle
../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs02499.pickle
../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs02999.pickle
../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs03499.pickle
../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs03999.pickle
../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs04499.pickle
../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs04999.pickle
../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs05499.pickle
../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs05999.pickle
../../bf_data/Nair_and_Abraham_2010/out1/Morphs/final_morphs06430.pickle


종종 fit 실패함. 이상한 object가 많으니 충분히 그럴 수 있긴 함. 눈으로 확인.

'모범적인' Sa, Sb, Sc, Sd가 잘 보이도록만 만들면 됨. fore/backgroud object 때문에 fit이 안 되는 경우는 그냥 무시. 얼마나 자주 그러는지만 체크. 


